In [ ]:
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from IPython.display import HTML

import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('features/legitimate_train.csv')
legi_train = df['url']
legiy_train = np.zeros(len(legi_train))

df = pd.read_csv('features/phish_train.csv')
phis_train = df['url']
phisy_train = np.ones(len(phis_train))

In [ ]:
df = pd.read_csv('features/legitimate_test.csv')
legi_test = df['url']
legiy_test = np.zeros(len(legi_test))

df = pd.read_csv('features/phish_test.csv')
phis_test = df['url']
phisy_test = np.ones(len(phis_test))

In [ ]:
x_train = np.append(legi_train,phis_train)
len_train = len(x_train)
x = np.append(x_train,legi_test)
x = np.append(x,phis_test)
y_train = np.append(legiy_train,phisy_train)
y = np.append(y_train,legiy_test)
y = np.append(y,phisy_test)

In [ ]:
import string
import numpy as np
from urllib.parse import urlparse
from sklearn.preprocessing import OneHotEncoder

# Define the function to extract relevant features from a URL
def extract_features(url):
    parsed_url = urlparse(url)
    domain = parsed_url.netloc
    path = parsed_url.path

    # Initialize feature vector with zeros
    features = [0] * (len(string.ascii_letters) + 10)

    # Add domain-based features
    for char in domain:
        if char.isalpha():
            features[ord(char.lower()) - ord('a')] += 1
        elif char.isdigit():
            features[ord(char) - ord('0') + 26] += 1
        else:
            features[-1] += 1

    # Add path-based features
    for char in path:
        if char.isalpha():
            features[ord(char.lower()) - ord('a') + 36] += 1
        elif char.isdigit():
            features[ord(char) - ord('0') + 52] += 1
        else:
            features[-2] += 1

    return features

# Define the preprocess function that converts a list of URLs to numerical form using one-hot encoding
def preprocess(urls):
    features = [extract_features(url) for url in urls]
    encoder = OneHotEncoder(categories='auto')
    encoder.fit(features)
    X = encoder.transform(features).toarray()
    return X

# # Example usage
# urls = ['https://www.google.com/search?q=phishing+website+detection',
#         'https://www.paypal.com/login',
#         'http://www.phishing.com']
X = preprocess(x)
print(X.shape)

(10550, 813)


In [ ]:
x_train = X[:len_train]
y_train = y[:len_train]

x_test = X[len_train:]
y_test = y[len_train:]

In [ ]:
x_train.size, x_test.size

(7734882, 842268)

In [ ]:
print(x_train)

[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]


In [ ]:
model = models.Sequential([
    layers.Embedding(input_dim=9514, output_dim=100, input_length=813),
    layers.Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.LSTM(128),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, batch_size = 32, epochs = 50)

Epoch 1/50
298/298 [==============================] - 157s 518ms/step - loss: 0.6759 - accuracy: 0.5529
Epoch 2/50
298/298 [==============================] - 155s 521ms/step - loss: 0.6139 - accuracy: 0.6613
Epoch 3/50
298/298 [==============================] - 154s 517ms/step - loss: 0.6051 - accuracy: 0.6743
Epoch 4/50
298/298 [==============================] - 154s 516ms/step - loss: 0.6017 - accuracy: 0.6761
Epoch 5/50
298/298 [==============================] - 154s 517ms/step - loss: 0.6007 - accuracy: 0.6777
Epoch 6/50
298/298 [==============================] - 155s 519ms/step - loss: 0.5952 - accuracy: 0.6871
Epoch 7/50
298/298 [==============================] - 154s 517ms/step - loss: 0.5890 - accuracy: 0.6913
Epoch 8/50
298/298 [==============================] - 155s 520ms/step - loss: 0.5862 - accuracy: 0.6932
Epoch 9/50
298/298 [==============================] - 155s 521ms/step - loss: 0.5852 - accuracy: 0.6964
Epoch 10/50
298/298 [==============================] - 154s 518m

In [ ]:
history = model.evaluate(x_test,y_test)

33/33 [==============================] - 17s 501ms/step - loss: 0.6077 - accuracy: 0.6641
